In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import math

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./6.type.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Type No'] = input_['Type No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            b = 0
            while True:
                b += 1
                
                try:
                    resp = requests.get(input_.loc[a, 'Type Url'],
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))
                    if resp.status_code == 200:
                        break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            try:
                count = int(html.xpath('//span[@id="lblResultCount"]/text()')[0].strip())
                if type(count) != int or count <= 0:
                    raise
            except:
                raise

            # = = = = = = = = = = = = = = =
            
            df_temp = pd.DataFrame([{'Type No': input_.loc[a, 'Type No'],
                                     'Type': input_.loc[a, 'Type'],
                                     'Type Url': input_.loc[a, 'Type Url'],
                                     'SKU Count': count,
                                     'Page': math.ceil(count / 100)}])

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Type No': input_.loc[a, 'Type No'],
                                     'Type': input_.loc[a, 'Type'],
                                     'Type Url': input_.loc[a, 'Type Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Type No']}.{input_.loc[a, 'Type']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Type No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./7.page-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Type No'],
                                            ascending=[False],
                                            ignore_index=True)
    output_error.to_excel('./page_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：2] - 1258.Liftgate Glass Hinge
[剩余数量：1000] - [当前时间：14:17:59]

[状态：ok，尝试次数：1] - 1269.Liftgate Pull Handle Clip
[剩余数量：999] - [当前时间：14:17:59]

[状态：ok，尝试次数：1] - 1271.Load Leveler Relay
[剩余数量：998] - [当前时间：14:18:00]

[状态：ok，尝试次数：1] - 1270.Liftgate Strap Repair Kit
[剩余数量：997] - [当前时间：14:18:00]

[状态：ok，尝试次数：1] - 1274.Locking Hub Service Kit
[剩余数量：996] - [当前时间：14:18:01]

[状态：ok，尝试次数：1] - 1266.Liftgate Lock Actuator
[剩余数量：995] - [当前时间：14:18:01]

[状态：ok，尝试次数：1] - 1272.Locking Hub
[剩余数量：994] - [当前时间：14:18:01]

[状态：ok，尝试次数：1] - 1267.Liftgate Object Sensor Retainer
[剩余数量：993] - [当前时间：14:18:02]

[状态：ok，尝试次数：1] - 1281.Manual Transmission Shift Lever Bushing
[剩余数量：992] - [当前时间：14:18:02]

[状态：ok，尝试次数：1] - 1254.License Plate Retainer
[剩余数量：991] - [当前时间：14:18:03]

[状态：ok，尝试次数：1] - 1280.Manual Transmission Shift Knob
[剩余数量：990] - [当前时间：14:18:03]

[状态：ok，尝试次数：1] - 1256.Liftgate Actuator
[剩余数量：989] - [当前时间：14:18:03]

[状态：ok，尝试次数：1] - 1282.Manual Transmission Shift Tower Kit
[剩余数量：988] - [当前时间：14: